In [1]:
import numpy as np 
import tensorflow as tf
import tensorflow_probability as tfp
import matplotlib.pyplot as plt


2023-03-07 16:13:47.696720: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-07 16:13:48.919638: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib/cuda/include:/usr/lib/cuda/lib64:/usr/lib/x86_64-linux-gnu/gazebo-11/plugins:/opt/ros/humble/opt/rviz_ogre_vendor/lib:/opt/ros/humble/lib/x86_64-linux-gnu:/opt/ros/humble/lib:/home/henri/ros2_humble/install/rosbag2_storage_mcap/lib:/home/henri/ros2_humble/install/rosbag2_compression_zstd/lib:/home/henri/ros2_humble/install/mcap_vendor/lib:/home/henri/ros2_humble/install/zstd_vendor/lib:/ho

In [16]:
class CAE(tf.keras.Model):
    def __init__(self, latent_dim):
        super(CAE, self).__init__()
        self.latent_dim = latent_dim
        self.encoder = tf.keras.Sequential(
            [
                tf.keras.layers.InputLayer(input_shape=(64,64 , 3)),
                tf.keras.layers.Conv2D(
                    filters=(32), kernel_size=3, strides=(2, 2), activation='relu'),
                tf.keras.layers.Conv2D(
                    filters=(64), kernel_size=3, strides=(2, 2), activation='relu'),
                tf.keras.layers.Conv2D(
                    filters=(64), kernel_size=3, strides=(2, 2), activation='relu'),
                tf.keras.layers.Conv2D(
                    filters=(64), kernel_size=3, strides=(2, 2), activation='relu'),
                tf.keras.layers.Flatten(),
                tf.keras.layers.Dense(16*16
                                        *32, activation=tf.nn.relu),
                # No activation
                tf.keras.layers.Dense(latent_dim),
            ])
        self.decoder = tf.keras.Sequential(
            [
                tf.keras.layers.InputLayer(input_shape=(latent_dim,)),
                tf.keras.layers.Dense(units=16*16*32, activation=tf.nn.relu),
                tf.keras.layers.Reshape(target_shape=(16, 16, 32)),
                tf.keras.layers.Conv2DTranspose(
                    filters=64, kernel_size=3, strides=2, padding='same',
                    activation='relu'),
                tf.keras.layers.Conv2DTranspose(
                    filters=64, kernel_size=3, strides=2, padding='same',
                ),
                tf.keras.layers.Conv2DTranspose(
                    filters=64, kernel_size=3, strides=2, padding='same',
                ),
                tf.keras.layers.Conv2D(
                    filters=32, kernel_size=3, strides=2, padding='same',
                ),
                # No activation
                tf.keras.layers.Conv2DTranspose(  
                    filters=3, kernel_size=3, strides=1, padding='same')
            ]
            )
        
    def encode(self, x,training=False):
        return self.encoder(x,training=training)
        
    def decode(self, x,training=False):
        return self.decoder(x,training=training)
        
    @tf.function
    def call(self, x,training=False):
        return self.decode(self.encode(x,training=training),training=training)
            
        
        


In [17]:
CAE(10)(np.arange(64*64*3).reshape(1,64,64,3)).shape

TensorShape([1, 64, 64, 3])